<a href="https://colab.research.google.com/github/gyunbao022/GamaLLM/blob/main/Gamma_LLM_%EB%AA%A8%EB%8D%B8_%EC%A0%9C%EC%9E%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 1. 최신 버전의 라이브러리 설치 (이 한 줄만 남깁니다)
!pip install -U transformers datasets accelerate peft bitsandbytes trl

# 2. Hugging Face 로그인
from huggingface_hub import notebook_login
notebook_login()

  Using cached trl-0.23.0-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.23.0-py3-none-any.whl (564 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.7.11
    Uninstalling trl-0.7.11:
      Successfully uninstalled trl-0.7.11


In [10]:
# (이전과 동일한 전체 파인튜닝 코드를 여기에 붙여넣습니다)
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import pandas as pd

# 1. 데이터셋 생성
data = [
    {"text": "### 지시: 의료 AI가 영상 진단 분야에서 하는 역할은?\n\n### 답변: 의료 AI는 엑스레이, CT, MRI와 같은 의료 영상에서 인간 의사가 놓칠 수 있는 미세한 병변이나 암세포를 높은 정확도로 탐지하여 질병의 조기 진단율을 높이는 데 기여합니다."},
    # ... (나머지 데이터 9개)
]
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

# 2. 모델 및 토크나이저 불러오기
model_name = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")

# 훈련 준비
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# 3. LoRA 설정
lora_config = LoraConfig(r=8, lora_alpha=32, lora_dropout=0.05, target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], bias="none", task_type="CAUSAL_LM")
model = get_peft_model(model, lora_config)

# 4. 훈련 설정
training_args = TrainingArguments(output_dir="./gemma-fine-tuned-model", per_device_train_batch_size=4, num_train_epochs=10, learning_rate=2e-4, logging_steps=10, bf16=True, report_to="none")

# 5. 훈련 시작
trainer = SFTTrainer(model=model, train_dataset=dataset, dataset_text_field="text", args=training_args, tokenizer=tokenizer)
print("Gemma 모델 파인튜닝을 시작합니다...")
trainer.train()
print("파인튜닝 완료!")

# 6. 모델 저장
trainer.save_model("./gemma-fine-tuned-model")
print("훈련된 모델이 'gemma-fine-tuned-model' 폴더에 저장되었습니다.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Adding EOS to train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Gemma 모델 파인튜닝을 시작합니다...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.297600
20,3.235300
30,3.289000


파인튜닝 완료!
훈련된 모델이 'gemma-fine-tuned-model' 폴더에 저장되었습니다.

--- 파인튜닝된 Gemma 모델 테스트 ---


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


### 지시: RAG 기술이 의료 AI에 왜 중요한가요?

### 답변: RAG 기술이 중요한 이유는 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음 다음


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 1. 원본 모델 및 토크나이저 불러오기
base_model_name = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# 2. 파인튜닝된 정보(어댑터) 불러와서 원본 모델에 결합하기
adapter_path = "./gemma-fine-tuned-model"
model = PeftModel.from_pretrained(base_model, adapter_path)

# 3. 테스트 프롬프트 작성
prompt = "### 지시: 신약 개발 과정에서 AI의 장점을 설명해줘.\n\n### 답변: 신약 개발 과정에서 AI의 장점은 다음과 같습니다."

# 4. 답변 생성
input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
output = model.generate(input_ids, max_new_tokens=100, repetition_penalty=1.2 )
response = tokenizer.decode(output[0], skip_special_tokens=True)

print("--- 전문가 AI의 답변 ---")
print(response)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

--- 전문가 AI의 답변 ---
### 지시: 신약 개발 과정에서 AI의 장점을 설명해줘.

### 답변: 신약 개발 과정에서 AI의 장점은 다음과 같습니다.

1. **신약 디자인 및 개발 단계를 자동화**: AI는 데이터 분석, 시스템 도구, 의료 데이터와 같은 다양한 정보를 활용하여 신약 디자인 및 개발 과정을 자동화할 수 있습니다.
2. **새로운 치료 방법 개발**: AI는 기존 치료 방법에 비해 효과적인 새로운 치료 방법을 개발하는 데 도움이 될 수 있습니다.
3. **투명성과 책
